In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/10 19:11:19 WARN Utils: Your hostname, LappyToppy, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/10 19:11:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/10 19:11:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/10 19:11:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
clubs = spark.read.parquet("/home/gnana/football-lakehouse/bronze/clubs")

In [4]:
clubs.limit(5).toPandas()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,None,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,None,2023,../data/raw/transfermarkt-scraper/2023/clubs.json.gz,https://www.transfermarkt.co.uk/sv-darmstadt-98/startseite/verein/105
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,None,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,None,2023,../data/raw/transfermarkt-scraper/2023/clubs.json.gz,https://www.transfermarkt.co.uk/ural-ekaterinburg/startseite/verein/11127
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,None,30,26.6,15,50.0,8,Beşiktaş Park,42445,€-25.26m,None,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/besiktas-istanbul/startseite/verein/114
3,12,as-rom,Associazione Sportiva Roma,IT1,None,26,26.3,18,69.2,17,Olimpico di Roma,70634,€-76.90m,None,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/as-rom/startseite/verein/12
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,None,30,25.5,21,70.0,18,Tottenham Hotspur Stadium,62850,€-120.05m,None,2024,../data/raw/transfermarkt-scraper/2024/clubs.json.gz,https://www.transfermarkt.co.uk/tottenham-hotspur/startseite/verein/148


In [5]:
clubs.printSchema()

root
 |-- club_id: integer (nullable = true)
 |-- club_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: string (nullable = true)
 |-- squad_size: integer (nullable = true)
 |-- average_age: double (nullable = true)
 |-- foreigners_number: integer (nullable = true)
 |-- foreigners_percentage: double (nullable = true)
 |-- national_team_players: integer (nullable = true)
 |-- stadium_name: string (nullable = true)
 |-- stadium_seats: integer (nullable = true)
 |-- net_transfer_record: string (nullable = true)
 |-- coach_name: string (nullable = true)
 |-- last_season: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- url: string (nullable = true)



In [9]:
clubs_silver = clubs.select(
    col("club_id"),
    col("name"),
    col("squad_size"),
    col("average_age"),
    col("coach_name")
).dropDuplicates(["club_id"])

In [10]:
clubs_silver.count()

439

In [11]:
clubs_silver.select("club_id").distinct().count()

439

In [13]:
clubs_silver.write.mode("overwrite").parquet("/home/gnana/football-lakehouse/silver/clubs_silver")